In [238]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [239]:
# 1. Veri Yükleme
data = pd.read_csv("male_players.csv")  # Verisetini yükle

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11044\3161535642.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("male_players.csv")  # Verisetini yükle


In [240]:
# 2. Eksik Verileri Temizleme
data = data.dropna(subset=['value_eur'])  # Hedef sütunda eksik veri varsa kaldır
data.fillna(0, inplace=True)  # Diğer eksik verileri 0 ile doldur

In [241]:
# 3. Kullanılmayacak Sütunları Kaldır
unused_columns = [
                    'fifa_version', 'fifa_update', 'update_as_of', '' 'player_id', 'player_url', 'short_name', 'long_name', 'dob', 'club_team_id', 'club_name', 
                    'league_id', 'league_name', 'club_position', 'club_jersey_number', 'club_loaned_from', 
                    'club_joined_date', 'club_contract_valid_until_year', 'nationality_id', 'nationality_name', 
                    'player_tags', 'player_traits','ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 
    'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 
    'lcb', 'cb', 'rcb', 'rb', 'gk','release_clause_eur', 'nation_team_id', 'nation_jersey_number','nation_position', 'body_type','league_level','real_face',]
data = data.drop(columns=unused_columns)

# Kullanılan sütunları yazdır
used_columns = data.columns
print("Kullanılan sütunlar:")
print(used_columns)

Kullanılan sütunlar:
Index(['player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'height_cm', 'weight_kg', 'preferred_foot', 'weak_foot',
       'skill_moves', 'international_reputation', 'work_rate', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_standing

In [242]:
# 4. "player_positions" sütunundaki değerlerden sadece ilk pozisyonu al
data['player_positions'] = data['player_positions'].apply(
    lambda x: x.split(',')[0].strip() if isinstance(x, str) else x
)

# 4.1 Kategorik Verileri Sayısal Değerlere Dönüştürme
categorical_columns = ['preferred_foot', 'work_rate', 'player_positions']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))  # Orijinal sütunu dönüştür
    label_encoders[col] = le

# 4.2 LabelEncoder karşılıklarını yazdır
for col, le in label_encoders.items():
    print(f"{col} için LabelEncoder değerleri:")
    label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    for key, value in label_mapping.items():
        print(f"  {key}: {value}")
    print("\n")

preferred_foot için LabelEncoder değerleri:
  Left: 0
  Right: 1


work_rate için LabelEncoder değerleri:
  High/High: 0
  High/Low: 1
  High/Medium: 2
  Low/High: 3
  Low/Low: 4
  Low/Medium: 5
  Medium/High: 6
  Medium/Low: 7
  Medium/Medium: 8


player_positions için LabelEncoder değerleri:
  CAM: 0
  CB: 1
  CDM: 2
  CF: 3
  CM: 4
  GK: 5
  LB: 6
  LM: 7
  LW: 8
  LWB: 9
  RB: 10
  RM: 11
  RW: 12
  RWB: 13
  ST: 14




In [243]:
print("Veri çerçevesindeki sütunlar:")
print(data.columns)

Veri çerçevesindeki sütunlar:
Index(['player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'height_cm', 'weight_kg', 'preferred_foot', 'weak_foot',
       'skill_moves', 'international_reputation', 'work_rate', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending

In [234]:
# ST pozisyonunun sayısal karşılığını al
st_label = label_encoders['player_positions'].transform(['ST'])[0]

# Sadece ST pozisyonundaki oyuncuları filtrele
st_data = data[data['player_positions'] == st_label]

# Özellik ve hedef ayrımı
X_st = st_data.drop(columns=['value_eur'])  # Özellikler
y_st = st_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_st, X_test_st, y_train_st, y_test_st = train_test_split(X_st, y_st, test_size=0.2, random_state=42)

In [235]:
# Model eğitimi ST
st_model = RandomForestRegressor(random_state=42, n_estimators=100)
st_model.fit(X_train_st, y_train_st)

RandomForestRegressor(random_state=42)

In [236]:
# Tahmin ve performans değerlendirme ST
y_pred_st = st_model.predict(X_test_st)
mae_st = mean_absolute_error(y_test_st, y_pred_st)
rmse_st = np.sqrt(mean_squared_error(y_test_st, y_pred_st))
r2_st = r2_score(y_test_st, y_pred_st)

print(f"ST Model Performansı:")
print(f"MAE: {mae_st}")
print(f"RMSE: {rmse_st}")
print(f"R^2: {r2_st}")

ST Model Performansı:
MAE: 368053.5130879748
RMSE: 1627152.274331881
R^2: 0.9404836828977303


In [237]:
# Feature importance ST
importances_st = st_model.feature_importances_
feature_names_st = X_st.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_st = np.argsort(importances_st)[::-1]

print("ST için Feature Importance:")
for i in sorted_indices_st:
    print(f"{feature_names_st[i]}: {importances_st[i]:.4f}")

ST için Feature Importance:
overall: 0.6772
potential: 0.1681
movement_sprint_speed: 0.0284
age: 0.0217
mentality_composure: 0.0151
movement_reactions: 0.0140
attacking_finishing: 0.0102
skill_dribbling: 0.0059
attacking_short_passing: 0.0048
defending_marking_awareness: 0.0046
movement_acceleration: 0.0037
defending: 0.0024
power_strength: 0.0022
attacking_heading_accuracy: 0.0022
wage_eur: 0.0021
dribbling: 0.0021
mentality_interceptions: 0.0020
pace: 0.0020
mentality_aggression: 0.0019
mentality_positioning: 0.0019
movement_agility: 0.0019
power_stamina: 0.0018
power_jumping: 0.0017
mentality_penalties: 0.0016
skill_long_passing: 0.0016
skill_fk_accuracy: 0.0016
shooting: 0.0016
attacking_crossing: 0.0014
physic: 0.0013
attacking_volleys: 0.0012
mentality_vision: 0.0012
power_shot_power: 0.0011
skill_ball_control: 0.0011
defending_sliding_tackle: 0.0010
skill_curve: 0.0009
preferred_foot: 0.0009
movement_balance: 0.0009
power_long_shots: 0.0009
defending_standing_tackle: 0.0007
weig

In [245]:
# GK pozisyonunun sayısal karşılığını al
gk_label = label_encoders['player_positions'].transform(['GK'])[0]

# Sadece GK pozisyonundaki oyuncuları filtrele
gk_data = data[data['player_positions'] == gk_label]

# Özellik ve hedef ayrımı
X_gk = gk_data.drop(columns=['value_eur'])  # Özellikler
y_gk = gk_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_gk, X_test_gk, y_train_gk, y_test_gk = train_test_split(X_gk, y_gk, test_size=0.2, random_state=42)

In [246]:
# Model eğitimi GK
gk_model = RandomForestRegressor(random_state=42, n_estimators=100)
gk_model.fit(X_train_gk, y_train_gk)

RandomForestRegressor(random_state=42)

In [247]:
# Tahmin ve performans değerlendirme GK
y_pred_gk = gk_model.predict(X_test_gk)
mae_gk = mean_absolute_error(y_test_gk, y_pred_gk)
rmse_gk = np.sqrt(mean_squared_error(y_test_gk, y_pred_gk))
r2_gk = r2_score(y_test_gk, y_pred_gk)

print(f"GK Model Performansı:")
print(f"MAE: {mae_gk}")
print(f"RMSE: {rmse_gk}")
print(f"R^2: {r2_gk}")

GK Model Performansı:
MAE: 248561.24968346418
RMSE: 1395478.363967396
R^2: 0.9408745010399016


In [248]:
# Feature importance GK
importances_gk = gk_model.feature_importances_
feature_names_gk = X_gk.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_gk = np.argsort(importances_gk)[::-1]

print("GK için Feature Importance:")
for i in sorted_indices_gk:
    print(f"{feature_names_gk[i]}: {importances_gk[i]:.4f}")

GK için Feature Importance:
overall: 0.6237
potential: 0.2603
age: 0.0367
power_shot_power: 0.0248
goalkeeping_reflexes: 0.0079
wage_eur: 0.0078
goalkeeping_positioning: 0.0030
goalkeeping_handling: 0.0024
movement_reactions: 0.0022
skill_dribbling: 0.0020
mentality_penalties: 0.0016
movement_agility: 0.0015
attacking_short_passing: 0.0014
mentality_composure: 0.0014
defending_marking_awareness: 0.0013
skill_ball_control: 0.0013
skill_curve: 0.0012
goalkeeping_diving: 0.0012
skill_long_passing: 0.0012
skill_fk_accuracy: 0.0011
goalkeeping_kicking: 0.0010
power_strength: 0.0010
goalkeeping_speed: 0.0009
mentality_interceptions: 0.0009
defending_standing_tackle: 0.0008
height_cm: 0.0008
power_jumping: 0.0008
movement_sprint_speed: 0.0008
mentality_vision: 0.0008
weight_kg: 0.0007
attacking_volleys: 0.0007
attacking_heading_accuracy: 0.0007
power_long_shots: 0.0007
power_stamina: 0.0007
movement_acceleration: 0.0007
mentality_positioning: 0.0007
movement_balance: 0.0006
attacking_finishin

In [ ]:
# 5. Özellik ve Hedef Ayrımı
X = data.drop(columns=['value_eur'])  # Özellikler
y = data['value_eur']  # Hedef

In [161]:
# 6. Eğitim ve Test Setlerine Ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [162]:
print("X_train sütun türleri:")
print(X_train.dtypes)

X_train sütun türleri:
player_positions                 int32
overall                          int64
potential                        int64
wage_eur                       float64
age                              int64
height_cm                        int64
weight_kg                        int64
preferred_foot                   int32
weak_foot                        int64
skill_moves                      int64
international_reputation         int64
work_rate                        int32
pace                           float64
shooting                       float64
passing                        float64
dribbling                      float64
defending                      float64
physic                         float64
attacking_crossing               int64
attacking_finishing              int64
attacking_heading_accuracy       int64
attacking_short_passing          int64
attacking_volleys                int64
skill_dribbling                  int64
skill_curve                      int64
sk

In [155]:
# 7. Model Eğitimi
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'ST'

In [70]:
# 8. Tahmin ve Performans Değerlendirme
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

MAE: 389930.47914207005
RMSE: 1423633.514466413
R^2: 0.9506555572261366


In [90]:
# 9. Feature Importance ve En Etkili 24 Özellik
importances = model.feature_importances_
feature_names = X.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices = np.argsort(importances)[::-1]
top_24_features = feature_names[sorted_indices[:24]]

print("En etkili 24 özellik:")
for i, feature in enumerate(top_24_features, 1):
    print(f"{i}. {feature} (Önem: {importances[sorted_indices[i-1]]:.4f})")

En etkili 24 özellik:
1. overall (Önem: 0.7053)
2. potential (Önem: 0.1668)
3. mentality_composure (Önem: 0.0260)
4. age (Önem: 0.0249)
5. movement_reactions (Önem: 0.0059)
6. movement_sprint_speed (Önem: 0.0034)
7. wage_eur (Önem: 0.0032)
8. attacking_finishing (Önem: 0.0032)
9. mentality_positioning (Önem: 0.0030)
10. pace (Önem: 0.0028)
11. movement_acceleration (Önem: 0.0027)
12. mentality_vision (Önem: 0.0025)
13. skill_dribbling (Önem: 0.0023)
14. power_shot_power (Önem: 0.0023)
15. mentality_penalties (Önem: 0.0023)
16. power_stamina (Önem: 0.0023)
17. movement_balance (Önem: 0.0019)
18. attacking_volleys (Önem: 0.0018)
19. skill_long_passing (Önem: 0.0018)
20. power_jumping (Önem: 0.0018)
21. player_positions (Önem: 0.0017)
22. defending_marking_awareness (Önem: 0.0017)
23. attacking_heading_accuracy (Önem: 0.0016)
24. skill_ball_control (Önem: 0.0016)


In [137]:
# LabelEncoder ile dönüştürülen sütunların değerlerini yazdır
for col, le in label_encoders.items():
    print(f"{col} için LabelEncoder değerleri:")
    label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    for key, value in label_mapping.items():
        print(f"  {key}: {value}")
    print("\n")

preferred_foot için LabelEncoder değerleri:
  0: 0
  1: 1


work_rate için LabelEncoder değerleri:
  0: 0
  1: 1
  2: 2
  3: 3
  4: 4
  5: 5
  6: 6
  7: 7
  8: 8


player_positions için LabelEncoder değerleri:
  0: 0
  1: 1
  2: 2
  3: 3
  4: 4
  5: 5
  6: 6
  7: 7
  8: 8
  9: 9
  10: 10
  11: 11
  12: 12
  13: 13
  14: 14
  15: 15
  16: 16
  17: 17
  18: 18
  19: 19
  20: 20
  21: 21
  22: 22
  23: 23
  24: 24
  25: 25
  26: 26
  27: 27
  28: 28
  29: 29
  30: 30
  31: 31
  32: 32
  33: 33
  34: 34
  35: 35
  36: 36
  37: 37
  38: 38
  39: 39
  40: 40
  41: 41
  42: 42
  43: 43
  44: 44
  45: 45
  46: 46
  47: 47
  48: 48
  49: 49
  50: 50
  51: 51
  52: 52
  53: 53
  54: 54
  55: 55
  56: 56
  57: 57
  58: 58
  59: 59
  60: 60
  61: 61
  62: 62
  63: 63
  64: 64
  65: 65
  66: 66
  67: 67
  68: 68
  69: 69
  70: 70
  71: 71
  72: 72
  73: 73
  74: 74
  75: 75
  76: 76
  77: 77
  78: 78
  79: 79
  80: 80
  81: 81
  82: 82
  83: 83
  84: 84
  85: 85
  86: 86
  87: 87
  88: 88
  89: 89


In [109]:
# 1. Oyuncuları pozisyonlarına göre ayır
defender_positions = ['CB', 'LB', 'RB', 'LWB', 'RWB', 'CM', 'CDM', 'CM, CDM']
forward_positions = ['ST', 'CF', 'LW', 'RW']

# LabelEncoder'dan pozisyonların sayısal karşılıklarını al
defender_labels = [label_encoders['player_positions'].transform([pos])[0] for pos in defender_positions]
forward_labels = [label_encoders['player_positions'].transform([pos])[0] for pos in forward_positions]

# Oyuncuları rollerine göre ayır
data['player_role'] = data['player_positions'].apply(
    lambda x: 'defender' if x in defender_labels else
              'forward' if x in forward_labels else
              'other'
)

# 2. Feature importance dictionary
feature_importance_dict = {
    'defender': {
        'defending_marking_awareness': 2,
        'defending_standing_tackle': 2,
        'defending_sliding_tackle': 2,
        'physic': 1.5,
        'mentality_interceptions': 1.5,
        'pace': 1.2
    },
    'forward': {
        'attacking_finishing': 2,
        'attacking_heading_accuracy': 1.8,
        'shooting': 1.5,
        'dribbling': 1.5,
        'movement_acceleration': 1.2,
        'movement_sprint_speed': 1.2
    }
}

# 3. Özellikleri ağırlıklandır
for role, importance_dict in feature_importance_dict.items():
    for feature, multiplier in importance_dict.items():
        data.loc[data['player_role'] == role, feature] *= multiplier

# 4. Kontrol için kullanılan sütunları yazdır
print("Ağırlıklandırılmış sütunlar:")
print(data.columns)

Ağırlıklandırılmış sütunlar:
Index(['player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'height_cm', 'weight_kg', 'preferred_foot', 'weak_foot',
       'skill_moves', 'international_reputation', 'work_rate', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_

In [115]:
# Rastgele bir oyuncu seç
sample_player = data.sample(1)

# Oyuncunun pozisyonunu ve rolünü yazdır
player_position = sample_player['player_positions'].values[0]
player_role = sample_player['player_role'].values[0]

print(f"Seçilen oyuncunun pozisyonu (LabelEncoded): {player_position}")
print(f"Seçilen oyuncunun rolü: {player_role}")

# Eğer pozisyonun string karşılığını görmek isterseniz:
decoded_position = label_encoders['player_positions'].inverse_transform([int(player_position)])[0]
print(f"Pozisyonun orijinal adı: {decoded_position}")

Seçilen oyuncunun pozisyonu (LabelEncoded): 549
Seçilen oyuncunun rolü: other
Pozisyonun orijinal adı: CM, CDM


In [96]:
# Rastgele bir futbolcu verisi oluştur
sample_player = {
    'player_positions': 1873,  # Örnek olarak '0' (LabelEncoder ile dönüştürülmüş bir değer)
    'overall': 55,
    'potential': 70,
    'wage_eur': 200000,
    'age': 27,
    'height_cm': 185,
    'weight_kg': 80,
    'preferred_foot': 1,  # Örnek olarak '1' (LabelEncoder ile dönüştürülmüş bir değer)
    'weak_foot': 4,
    'skill_moves': 4,
    'international_reputation': 3,
    'work_rate': 2,  # Örnek olarak '2' (LabelEncoder ile dönüştürülmüş bir değer)
    'pace': 85,
    'shooting': 80,
    'passing': 78,
    'dribbling': 82,
    'defending': 60,
    'physic': 75,
    'attacking_crossing': 70,
    'attacking_finishing': 85,
    'attacking_heading_accuracy': 75,
    'attacking_short_passing': 80,
    'attacking_volleys': 78,
    'skill_dribbling': 85,
    'skill_curve': 80,
    'skill_fk_accuracy': 75,
    'skill_long_passing': 78,
    'skill_ball_control': 85,
    'movement_acceleration': 85,
    'movement_sprint_speed': 84,
    'movement_agility': 80,
    'movement_reactions': 85,
    'movement_balance': 78,
    'power_shot_power': 85,
    'power_jumping': 75,
    'power_stamina': 80,
    'power_strength': 78,
    'power_long_shots': 80,
    'mentality_aggression': 70,
    'mentality_interceptions': 60,
    'mentality_positioning': 85,
    'mentality_vision': 80,
    'mentality_penalties': 75,
    'mentality_composure': 85,
    'defending_marking_awareness': 85,
    'defending_standing_tackle': 90,
    'defending_sliding_tackle': 85
}

sample_player_df = pd.DataFrame([sample_player])

# Model ile tahmin yap
predicted_value = model.predict(sample_player_df)
print(f"Tahmin edilen piyasa değeri (value_eur): €{predicted_value[0]:,.2f}")

Tahmin edilen piyasa değeri (value_eur): €260,700.00


In [78]:
sample_player = {
'player_positions': 1873, # Örneğin 'ST' pozisyonuna karşılık gelen kod
'overall': 88,
'potential': 90,
'wage_eur': 300000,
'age': 27,
'height_cm': 185,
'weight_kg': 80,
'preferred_foot': 1, # 1 = Sağ
'weak_foot': 4,
'skill_moves': 4,
'international_reputation': 4,
'work_rate': 2, # Örneğin "High/Low"
# Genel yetenekler
'pace': 92,
'shooting': 91,
'passing': 75,
'dribbling': 89,
'defending': 25,
'physic': 85,

# Attacking Attributes
'attacking_crossing': 65,
'attacking_finishing': 94,
'attacking_heading_accuracy': 87,
'attacking_short_passing': 78,
'attacking_volleys': 88,

# Skill Attributes
'skill_dribbling': 90,
'skill_curve': 83,
'skill_fk_accuracy': 70,
'skill_long_passing': 68,
'skill_ball_control': 89,

# Movement Attributes
'movement_acceleration': 90,
'movement_sprint_speed': 93,
'movement_agility': 84,
'movement_reactions': 89,
'movement_balance': 80,

# Power Attributes
'power_shot_power': 91,
'power_jumping': 85,
'power_stamina': 82,
'power_strength': 84,
'power_long_shots': 86,

# Mentality Attributes
'mentality_aggression': 68,
'mentality_interceptions': 22,
'mentality_positioning': 92,
'mentality_vision': 79,
'mentality_penalties': 88,
'mentality_composure': 90,

# Defending Attributes
'defending_marking_awareness': 18,
'defending_standing_tackle': 20,
'defending_sliding_tackle': 15
}

sample_player_df = pd.DataFrame([sample_player])

# Model ile tahmin yap
predicted_value = model.predict(sample_player_df)
print(f"Tahmin edilen piyasa değeri (value_eur): €{predicted_value[0]:,.2f}")

Tahmin edilen piyasa değeri (value_eur): €87,035,000.00


In [89]:
mbappe = {
'player_positions': 204, # LW (örneğin), LabelEncoder karşılığı
'overall': 91,
'potential': 94,
'wage_eur': 2_300_000,
'age': 40,
'height_cm': 182,
'weight_kg': 10,
'preferred_foot': 1, # Right: 1
'weak_foot': 4,
'skill_moves': 5,
'international_reputation': 5,
'work_rate': 1, # High/Low → örneğin 0
'pace': 97,
'shooting': 90,
'passing': 80,
'dribbling': 92,
'defending': 36,
'physic': 78,

# Attacking Attributes
'attacking_crossing': 78,
'attacking_finishing': 94,
'attacking_heading_accuracy': 73,
'attacking_short_passing': 86,
'attacking_volleys': 84,

# Skill Attributes
'skill_dribbling': 93,
'skill_curve': 80,
'skill_fk_accuracy': 69,
'skill_long_passing': 71,
'skill_ball_control': 92,

# Movement Attributes
'movement_acceleration': 97,
'movement_sprint_speed': 97,
'movement_agility': 93,
'movement_reactions': 93,
'movement_balance': 82,

# Power Attributes
'power_shot_power': 90,
'power_jumping': 88,
'power_stamina': 88,
'power_strength': 77,
'power_long_shots': 83,

# Mentality Attributes
'mentality_aggression': 64,
'mentality_interceptions': 38,
'mentality_positioning': 93,
'mentality_vision': 83,
'mentality_penalties': 84,
'mentality_composure': 88,

# Defending Attributes
'defending_marking_awareness': 2,
'defending_standing_tackle': 4,
'defending_sliding_tackle': 5
}

sample_player_df = pd.DataFrame([mbappe])

# Model ile tahmin yap
predicted_value = model.predict(sample_player_df)
print(f"Tahmin edilen piyasa değeri (value_eur): €{predicted_value[0]:,.2f}")

Tahmin edilen piyasa değeri (value_eur): €158,480,000.00
